In [ ]:
#default_exp encoder

In [ ]:
#hide
from nbdev.showdoc import *

# Encoder


This is just a wrapper around CLIP functions. Cool thing here is we can use the one model for both image and text!


In [ ]:
#export
import torch
import clip
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, _ = clip.load("ViT-B/32", device, jit=False)    
model = model.float()

In [ ]:
#export
def image_encoder(img_loader, device):
    image_embeddings = torch.tensor(()).to(device)
    with torch.no_grad():
        for images, labels in tqdm(img_loader):
            batch_features = model.encode_image(images)
            image_embeddings = torch.cat((image_embeddings, batch_features)).to(device)
    
    image_embeddings = image_embeddings / image_embeddings.norm(dim=-1, keepdim=True)
    return(image_embeddings)

In [ ]:
new_files = [('images/Wholesome-Meme-8.jpg', 'Wholesome-Meme-8'), ('images/Wholesome-Meme-1.jpg', 'Wholesome-Meme-1')]

In [ ]:
from memery.crafter import crafter

In [ ]:
img_loader = crafter(new_files, device)

In [ ]:
model = model.float()
model.dtype

torch.float32

In [ ]:
[batch for batch in img_loader][0][0].dtype


torch.float32

In [ ]:
image_embeddings = image_encoder(img_loader, device)

  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same

In [ ]:
image_embeddings.shape

torch.Size([2, 512])

The text encoder returns a 512d vector just like the image encoder

In [ ]:
#export
def text_encoder(text, device):
    with torch.no_grad():
        text = clip.tokenize(text).to(device)
        text_features = model.encode_text(text)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    return(text_features)

In [ ]:
text_embedding = text_encoder('a funny dog meme', device)

In [ ]:
text_embedding.shape

torch.Size([1, 512])

In [ ]:
#export
def image_query_encoder(image, device):
    with torch.no_grad():
        image_embed = model.encode_image(image.unsqueeze(0))
    image_embed = image_embed / image_embed.norm(dim=-1, keepdim=True)
    return(image_embed)